# Example using blaze-server

In this example, use the `blaze-server` to serve iris data from different data source. The data source is a CSV file, a table in an sqlite DB and from a table in a postgresql DB. Use the blaze-server to share the datasets.

 * [Add iris data to postgresql and add source to blaze-server](#Add-iris-data-to-postgresql)
 * [Define config file and serve data](#Command-Line-Tool:-blaze-server)
   * [YAML config](#YAML-Specification)
   * [Start server](#Start-server)
 * [Explore data](#Explore-data)
    * [Available datasets](#Datasets-served-by-blaze-server)
    * [`peek()` into data]()

## Add `iris` data to postgresql

__Q: Can we not get the `d.iriscsv` into a DataFrame, then add it to postgresql?__
<br/>Example:  `df = bz.compute(d.iriscsv, output_type=pd.DataFrame)` - this didn't work.


Postgresql can be conda installed from blaze channel.
```sh
    > conda install -c blaze postgresql=9.3.5
```

1. Ensure you have a postgres DB running.
2. Following steps assume postgresql is running locally and you can log into the DB without requiring a username.
3. Create a DB named `tutorial` - we will add the data to a table called `iris` 
```sh
    > createdb tutorial
```
4. Run the cells below to add iris data to DB in a table called iris

In [1]:
import blaze as bz
import pandas as pd

In [2]:
iris_df = bz.compute(bz.data(bz.utils.example('iris.csv')), output_type=pd.DataFrame)
t = bz.odo(iris_df, 'postgresql://localhost/tutorial::iris')

## Command Line Tool: blaze-server 

Blaze provides uniform access to a variety of common data formats.  Blaze
Server builds off of this uniform interface to host data remotely through a
JSON web API.

Blaze ships with a command line tool called ``blaze-server`` to serve up data
sources specified in a YAML file.

__NOTE:__
```
   To use the YAML specification feature of Blaze server please install
   the `pyyaml` library. This can be done easily with ``conda``:

       conda install pyyaml
```

### YAML Specification

The structure of the specification file is as follows:

  ```yaml
     name1:
       source: path or uri
       dshape: optional datashape
     name2:
       source: path or uri
       dshape: optional datashape
     ...
     nameN:
       source: path or uri
       dshape: optional datashape
   ```

Use examples provided with blaze package to define specification file. Get the full path to each of the resouces and write to `conf.yaml`:

  ```python
    
    from blaze.utils import example
    lines = ['iriscsv: ',
             '  source: ' +  example('iris.csv'),
             'irisdb: ',
             '  source: ' + 'sqlite:///' + example('iris.db') + '::iris',
             'accounts: ',
             '  source: ' + example('accounts.json.gz'),
             '  dshape: ' + '"var * {name: string, amount: float64}"',
             'iris_psql: ',
             '  source: ' + 'postgresql://localhost/tutorial::iris']

    with open('conf.yaml', 'w') as f:
        f.write("\n".join(lines))
  ```

### Start server

In a command line terminal, start the blaze-server:

  ```sh
    blaze-server conf.yaml
  ```

### Explore data

In [3]:
d = bz.data('blaze://localhost:6363')

#### Datasets served by blaze-server ####
Looking at the fields and dshape attribute, we see there are 3 datasets being served and the types contained within each.
 * accounts
 * iriscsv
 * irisdb

In [4]:
print('Datasets:', d.fields)

print('\n')

print('dshape: ')
print(d.dshape)

Datasets: ['accounts', 'iris_psql', 'iriscsv', 'irisdb']


dshape: 
{
  accounts: 5 * {amount: int64, name: string},
  iris_psql: var * {
    sepal_length: float64,
    sepal_width: float64,
    petal_length: float64,
    petal_width: float64,
    species: ?string
    },
  iriscsv: var * {
    sepal_length: float64,
    sepal_width: float64,
    petal_length: float64,
    petal_width: float64,
    species: ?string
    },
  irisdb: var * {
    sepal_length: ?float64,
    sepal_width: ?float64,
    petal_length: ?float64,
    petal_width: ?float64,
    species: ?string
    }
  }


#### Peek into each dataset ####
 * each dataset can be accessed as an attribute or as a dict key
 * use ``peek()`` to look at the data which returns the tabular data as a pandas dataframe
 * use ``sample(n=).peek()`` to peek at a randomly subsampled bit of data. `n` referes to number of rows.
 * note the `irisdb` is a DB so it may contain multiple tables. In this case, it contains a single table called `iris`. <br/>This is accessed in the same way: either as an attribute or as a dict key.

In [5]:
print('peek() returns a pandas dataframe:')
print(type(d.accounts.peek()))

print('\n')

print('accounts dataset:')
print(d.accounts.peek())

print('\n')

print('iriscsv dataset:')
print(d['iriscsv'].sample(n=4).peek())

print('\n')

print('irisdb dataset access as dict keys:')
print(d['irisdb'].sample(n=4).peek())

print('\n')

print('iris_psql dataset access as attributes:')
print(d.iris_psql.sample(n=4).peek())

peek() returns a pandas dataframe:
<class 'pandas.core.frame.DataFrame'>


accounts dataset:
   amount     name
0     100    Alice
1    -200      Bob
2     300  Charlie
3     400   Dennis
4    -500    Edith


iriscsv dataset:
   sepal_length  sepal_width  petal_length  petal_width          species
0           5.7          4.4           1.5          0.4      Iris-setosa
1           6.8          3.2           5.9          2.3   Iris-virginica
2           5.9          3.2           4.8          1.8  Iris-versicolor
3           5.8          2.7           5.1          1.9   Iris-virginica


irisdb dataset access as dict keys:
   sepal_length  sepal_width  petal_length  petal_width          species
0           6.3          3.3           6.0          2.5   Iris-virginica
1           6.9          3.1           4.9          1.5  Iris-versicolor
2           4.8          3.4           1.9          0.2      Iris-setosa
3           6.8          3.2           5.9          2.3   Iris-virginica


iris

In [6]:
# Issue here: output_type doesn't work with data() interface when using compute()
# bz.compute(d.iris_psql.sample(n=10), output_type=pd.DataFrame)
bz.odo(d.iris_psql.sample(n=4), pd.DataFrame)

,sepal_length,sepal_width,petal_length,petal_width,species
0,6.7,3.1,4.7,1.5,Iris-versicolor
1,6.7,2.5,5.8,1.8,Iris-virginica
2,7.2,3.6,6.1,2.5,Iris-virginica
3,5.4,3.4,1.7,0.2,Iris-setosa


In [7]:
d.irisdb[d.irisdb['species'] == 'Iris-setosa'].sample(n=4).peek()

,sepal_length,sepal_width,petal_length,petal_width,species
0,4.8,3.4,1.9,0.2,Iris-setosa
1,5.7,4.4,1.5,0.4,Iris-setosa
2,4.6,3.6,1.0,0.2,Iris-setosa
3,5.4,3.4,1.7,0.2,Iris-setosa


In [8]:
# s = bz.symbol('s', bz.discover(d.iris_psql))
#bz.concat(d.irisdb['species'], d.irisdb['sepal_width'], axis=0)

df1 = bz.odo(d.irisdb[d.irisdb['species'] == 'Iris-setosa'].sample(n=4), pd.DataFrame)
df2 = bz.odo(d.iris_psql[d.iris_psql['species'] == 'Iris-versicolor'].sample(n=4), pd.DataFrame)
print(pd.concat((df1, df2)))

# Q: blaze concat is not working - not sure how to illustrate working with heterogenous data
# bz.concat(d.irisdb[d.irisdb['species'] == 'Iris-setosa'],
#           d.irisdb[d.irisdb['species'] == 'Iris-versicolor'])

   sepal_length  sepal_width  petal_length  petal_width          species
0           5.3          3.7           1.5          0.2      Iris-setosa
1           5.4          3.4           1.7          0.2      Iris-setosa
2           4.9          3.0           1.4          0.2      Iris-setosa
3           5.4          3.9           1.3          0.4      Iris-setosa
0           6.0          2.9           4.5          1.5  Iris-versicolor
1           5.7          3.0           4.2          1.2  Iris-versicolor
2           5.6          2.5           3.9          1.1  Iris-versicolor
3           6.5          2.8           4.6          1.5  Iris-versicolor


In [9]:
df1 = bz.odo(d.irisdb[d.irisdb['species'] == 'Iris-setosa'].sample(n=4), pd.DataFrame)
df2 = bz.odo(d.irisdb[d.irisdb['species'] == 'Iris-versicolor'].sample(n=4), pd.DataFrame)

pd.concat((df1, df2))

,sepal_length,sepal_width,petal_length,petal_width,species
0,5.0,3.0,1.6,0.2,Iris-setosa
1,4.8,3.0,1.4,0.3,Iris-setosa
2,5.5,3.5,1.3,0.2,Iris-setosa
3,5.4,3.9,1.7,0.4,Iris-setosa
0,5.6,2.9,3.6,1.3,Iris-versicolor
1,6.2,2.9,4.3,1.3,Iris-versicolor
2,5.8,2.7,3.9,1.2,Iris-versicolor
3,6.9,3.1,4.9,1.5,Iris-versicolor
